In [111]:
import pandas as pd
import numpy as np
import altair as alt

## Data ingestion and formating

In [112]:
df = pd.read_csv('../data/emotion_forced_choice_uw_students.csv')
df_labels = pd.read_csv('../data/emotion_labels.csv')

In [113]:
df = df.iloc[5:, :] # filter out test rows

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 5 to 85
Columns: 220 entries, StartDate to Q195.1
dtypes: object(220)
memory usage: 139.3+ KB


In [115]:
df = df[df['Finished'] =='True'] # filter out incomplete surveys

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 5 to 85
Columns: 220 entries, StartDate to Q195.1
dtypes: object(220)
memory usage: 88.1+ KB


In [117]:
df.rename(columns={
    'Q1.2': 'sex',
    'Q1.3_1': 'age',
    'Q1.4': 'ethnicity',
    'Q1.5': 'formal education',
    'Q1.6_1': 'income'}, inplace=True)

In [118]:
df.to_csv('../clean_data/forced_choice_emotion_uw_students.csv', index=False)

### Words by frequency for all images (ranking)

In [119]:
def count_freq_labels(df, X="all" ):
    if X == "all":
        df_counts = df.stack().reset_index(drop=True).value_counts() # stack as series
        df_counts = df_counts.to_frame('counts') # get value_counts as df
        df_counts['emotion'] = df_counts.index # get index as col
    else:
        df_counts = df[X].reset_index(drop=True).value_counts() # stack as series
        df_counts = df_counts.to_frame('counts') # get value_counts as df
        df_counts[X] = df_counts.index # get index as col

    df_counts = df_counts.reset_index(drop=True) # clean index
    df_counts['percent'] = df_counts['counts'] / df_counts['counts'].sum() # compute percentage
    return df_counts

In [120]:
def simple_per_bar(
    df, title='Title', X='percent:Q', Y='emotion:N', \
    width=450, height=250, sort='-x', \
    text_size = 12, label_size = 11, title_size=12, \
    emotion='Some', color1='#0570b0', color2='orange'):
    
    bars = alt.Chart(df, title=title).mark_bar().encode(
        alt.X(X, axis=alt.Axis(format='.0%')),
        y=alt.Y(Y, sort=sort), 
        color=alt.condition(
            alt.datum.emotion == emotion,
            alt.value(color2),
            alt.value(color1)
        ))
    
    text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3,  # Nudges text to right so it doesn't appear on top of the bar
    fontSize=text_size
    ).encode(
        alt.Text(X, format='.1%')
    )
    
    chart = (bars + text).configure_axis(
            labelFontSize=label_size,
            titleFontSize=title_size).properties(
                width=width, 
                height=height).display(renderer='svg')
    
    
    return chart

In [121]:
def simple_count_bar(
    
    df, title='Title', X='counts:Q', Y='emotion:N', \
    width=450, height=250, sort='-x', \
    text_size = 12, label_size = 11, title_size=12,
    emotion='Some', color1='#0570b0', color2='#orange'):
    
    bars = alt.Chart(df, title=title).mark_bar().encode(
        alt.X(X),
        y=alt.Y(Y, sort=sort), 
        color=alt.condition(
            alt.datum.emotion == emotion,
            alt.value(color2),
            alt.value(color1)
        ))
    
    text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3,  # Nudges text to right so it doesn't appear on top of the bar
    fontSize=text_size
    ).encode(
        alt.Text(X)
    )
    
    chart = (bars + text).configure_axis(
            labelFontSize=label_size,
            titleFontSize=title_size).properties(
                width=width, 
                height=height).display(renderer='svg')
    
    
    return chart

## Demographics

In [122]:
source = count_freq_labels(df, X="sex") 
title = 'Sex | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'sex:N'
w, h= 450, 100
txs, ls, ts = 12, 12, 12

chart_sex = simple_per_bar(source, title=title, X=X, Y=Y, width=w, height=h, text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

In [123]:
source = count_freq_labels(df, X="age") 
title = 'Age | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'age:N'
w, h= 450, 150
txs, ls, ts = 12, 11, 12

chart_age = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h, \
    text_size = txs, label_size = ls, title_size=ts)
chart_age

alt.LayerChart(...)

In [124]:
source = count_freq_labels(df, X="ethnicity") 
title = 'Ethnicity | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'ethnicity:N'
w, h= 450, 150
txs, ls, ts = 12, 11, 12

chart_ethnicity= simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h, \
    text_size = txs, label_size = ls, title_size=ts)
chart_ethnicity

alt.LayerChart(...)

In [125]:
source = count_freq_labels(df, X="formal education") 
title = 'Formal education | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'formal education:N'
w, h= 450, 150
txs, ls, ts = 12, 11, 12

chart_formal_education= simple_per_bar(
    source, title=title, X=X, Y=Y, width=w, height=h, text_size = txs, label_size = ls, title_size=ts)
chart_formal_education

alt.LayerChart(...)

## Overall results

In [190]:
df_emo_answers = df.loc[:, 'Q2.1':'Q195.1'] # subset photos
df_emo_answers.columns.to_frame().reset_index(drop=True).to_csv('../clean_data/photo_ids.csv', index=False)

In [127]:
source = count_freq_labels(df_emo_answers, X="all") 

# save list of emotions words to avoid repeating this computation later
emotion_words_list = source['emotion'].str.lower().tolist()
emotion_words_list.remove('other')

title = 'Labels frequency | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12

chart_overall_per = simple_per_bar(
    source, title=title, X=X, Y=Y, width=w, height=h, text_size = txs, label_size = ls, title_size=ts)
chart_overall_per

alt.LayerChart(...)

In [128]:
source = count_freq_labels(df_emo_answers, X="all") 
title = 'Labels frequency | n = '+ source['counts'].sum().astype(str)
X, Y = 'counts:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12

chart_overall_count = simple_count_bar(
    source, title=title, X=X, Y=Y, width=w, height=h, text_size = txs, label_size = ls, title_size=ts)
chart_overall_count

alt.LayerChart(...)

## Most frequently used word for each emotion category (grouping pictures by expected-emotion)

In [129]:
def emotion_df_formated(df_emo_answers, emotion_label):
    df_emo_cat = df_emo_answers.copy() 
    df_emo_cat_t = df_emo_cat.T # transpose
    df_emo_cat_t['photo_id'] = df_emo_cat_t.index # get index as col
    df_emo_cat_t = df_emo_cat_t.reset_index(drop=True) # clean index
    df_emo_cat_t_labels = pd.concat([df_emo_cat_t, df_labels], axis=1) # add metadata cols
    df_label =  df_emo_cat_t_labels[df_emo_cat_t_labels['label'] == emotion_label]
    return df_label

In [130]:
emotion = 'anger'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_anger = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h, emotion=emotion.capitalize(), \
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Disgust

In [131]:
emotion = 'disgust'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_disgust = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Fear

In [132]:
emotion = 'fear'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_fear = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Surprise

In [133]:
emotion = 'surprise'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_surprise = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(), \
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Happiness

In [134]:
emotion = 'happiness'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_happiness = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Sadness

In [135]:
emotion = 'sadness'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_sadness = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Uncertain

In [136]:
emotion = 'uncertain'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_uncertain = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

### Neutral

In [137]:
emotion = 'neutral'
df_formated = emotion_df_formated(df_emo_answers, emotion) # subset 'anger' rows
df_formated_ans = df_formated.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
source = count_freq_labels(df_formated_ans, X="all") 
title = 'Expcted label: '+ emotion + ' | n = '+ source['counts'].sum().astype(str)
X, Y = 'percent:Q', 'emotion:N'
w, h= 450, 200
txs, ls, ts = 12, 11, 12


chart_neutral = simple_per_bar(
    source, title=title, X=X, Y=Y, \
    width=w, height=h,  emotion=emotion.capitalize(),\
    text_size = txs, label_size = ls, title_size=ts)

alt.LayerChart(...)

## Most frequently used word for each emotion category (grouping pictures by expected-emotion), by ethnicity group

In [138]:
def emotion_df_formated_et(df_emo_answers, emotion_label, ethnicity):
    df_emo_cat = df_emo_answers.copy() 
    df_emo_cat_t = df_emo_cat.T # transpose
    df_emo_cat_t['photo_id'] = df_emo_cat_t.index # get index as col
    df_emo_cat_t = df_emo_cat_t.reset_index(drop=True) # clean index
    df_emo_cat_t_labels = pd.concat([df_emo_cat_t, df_labels], axis=1) # add metadata cols
    df_label =  df_emo_cat_t_labels[(df_emo_cat_t_labels['label'] == emotion_label) & (df_emo_cat_t_labels['ethnicity'] == ethnicity)]
    return df_label

In [139]:
def wrapper_chart_emotion(df_emo_answers, emotion, ethnicity):
    df = emotion_df_formated_et(df_emo_answers, emotion,  ethnicity) # subset 'anger' rows
    df_formated_ans = df.drop(['photo_id', 'ethnicity', 'sex','age', 'label', 'url'], axis=1)
    df_count = count_freq_labels(df_formated_ans) # count label freq
    chart = simple_per_bar(
        df_count, \
        title='Expected label: '+ emotion + ' | n = '+ df_count['counts'].sum().astype(str), \
         emotion=emotion.capitalize())
    return chart

## Anger

In [140]:
chart_anger_bipoc = wrapper_chart_emotion(df_emo_answers, 'anger', 'bipoc')
chart_anger_white = wrapper_chart_emotion(df_emo_answers, 'anger', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Disgust

In [141]:
chart_disgust_bipoc = wrapper_chart_emotion(df_emo_answers, 'disgust', 'bipoc')
chart_disgust_white = wrapper_chart_emotion(df_emo_answers, 'disgust', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Fear

In [142]:
chart_fear_bipoc = wrapper_chart_emotion(df_emo_answers, 'fear', 'bipoc')
chart_fear_white = wrapper_chart_emotion(df_emo_answers, 'fear', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Surprise

In [143]:
chart_surprise_bipoc= wrapper_chart_emotion(df_emo_answers, 'surprise', 'bipoc')
chart_surprise_white = wrapper_chart_emotion(df_emo_answers, 'surprise', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Happiness

In [144]:
chart_happiness_bipoc = wrapper_chart_emotion(df_emo_answers, 'happiness', 'bipoc')
chart_happiness_white = wrapper_chart_emotion(df_emo_answers, 'happiness', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Sadness

In [145]:
chart_sadness_bipoc = wrapper_chart_emotion(df_emo_answers, 'sadness', 'bipoc')
chart_sadness_white = wrapper_chart_emotion(df_emo_answers, 'sadness', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Neutral

In [146]:
chart_neutral_bipoc = wrapper_chart_emotion(df_emo_answers, 'neutral', 'bipoc')
chart_neutral_white = wrapper_chart_emotion(df_emo_answers, 'neutral', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

### Uncertain/Other

In [147]:
chart_uncertain_bipoc = wrapper_chart_emotion(df_emo_answers, 'uncertain', 'bipoc')
chart_uncertain_white = wrapper_chart_emotion(df_emo_answers, 'uncertain', 'white')

alt.LayerChart(...)

alt.LayerChart(...)

## Emotion percentages as feature vectors

In [148]:
# placeholder dataframe
df_emo_features = pd.DataFrame(columns = emotion_words_list) 

# add key-ids
df_emo_features.insert(loc=0, column='photo_id', value=df_emo_answers.columns) 

In [149]:
df_emo_answers = df.loc[:, 'Q2.1':'Q195.1'] # subset photos

In [150]:
def fill_in_emotion_vectors(df_emo_features, df_emo_answers, emotion_words_list):
    for emo in range(0, len(emotion_words_list)):
        
        emotion = emotion_words_list[emo] # get emotion word
        df_ans = emotion_df_formated(df_emo_answers, emotion) # ans for all pics of emo-word
        photo_id_list = df_ans['photo_id'].tolist() # photo ids for the emotion word
        splited_photo_id = np.array_split(photo_id_list, 24) # photo_id_list as an array 
        
        for ids in range(0, len(splited_photo_id)):
            photo_id = splited_photo_id[ids]  
            df_single = df_ans[df_ans['photo_id'].isin(photo_id)] # get row for nth photo
            df_emotion_ans = df_single.drop(['photo_id', 'ethnicity', 'sex', \
                                             'age', 'label', 'url'], axis=1) # clean up for calculation
            source = count_freq_labels(df_emotion_ans, X="all") # compute vector-values
            source = source[['percent', 'emotion']].T
            source.columns = source.iloc[1].str.lower() # emotion words as cols
            source.drop('emotion', axis=0, inplace=True) # clean up emotion row
            source.reset_index(drop=True, inplace=True) # clean up index
            source.columns.name = None
            photo_id_str = photo_id[0] # photo id as str
            source.insert(0, 'photo_id', photo_id_str) # photo id as col
            # set photo id as col for easier manipulation
            df_emo_features = df_emo_features.set_index('photo_id') 
            # fill in vectors where the value is available
            df_emo_features.loc[photo_id_str] = pd.Series(source.T.to_dict()[0])
            
            df_emo_features['photo_id'] = df_emo_features.index # index back to column
            df_emo_features.index.name = None 
            df_emo_features.reset_index(drop=True, inplace=True) # clean up index
    
    return df_emo_features 
            


In [151]:
df_emo_vectors = fill_in_emotion_vectors(df_emo_features, df_emo_answers, emotion_words_list)

In [152]:
df_emo_vectors = df_emo_vectors.fillna(0)

In [153]:
df_emo_vectors = pd.concat([df_emo_vectors, df_labels], axis=1) # concat metadata

In [154]:
df_emo_vectors.to_csv('../clean_data/forced_choice_uw_students_vectors.csv', index=False)

# Clustering

In [159]:
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import preprocessing as pp

## K means

### Standarize feature matrix X

In [160]:
X = df_emo_vectors.copy().drop(['photo_id', 'ethnicity', 'sex', \
                                   'age', 'label', 'url'], axis=1)

In [161]:
features = X.columns
sX = pp.StandardScaler(copy=True)
X.loc[:,features] = sX.fit_transform(X[features])

In [162]:
X.describe() # mean  = 0, SD =1

,happiness,neutral,surprise,sadness,disgust,anger,fear,uncertain
count,1.940000e+02,1.940000e+02,1.940000e+02,1.940000e+02,1.940000e+02,1.940000e+02,1.940000e+02,1.940000e+02
mean,-5.264975e-17,2.323456e-16,-3.490907e-17,5.722799e-19,3.862889e-17,-3.777047e-17,7.153499e-19,-7.325183e-17
std,1.002587e+00,1.002587e+00,1.002587e+00,1.002587e+00,1.002587e+00,1.002587e+00,1.002587e+00,1.002587e+00
min,-5.276933e-01,-6.218201e-01,-5.762526e-01,-4.810820e-01,-5.701140e-01,-5.196366e-01,-5.261521e-01,-1.010593e+00
25%,-5.276933e-01,-6.218201e-01,-5.762526e-01,-4.810820e-01,-5.701140e-01,-5.196366e-01,-5.261521e-01,-7.085053e-01
50%,-4.614328e-01,-4.936688e-01,-4.930113e-01,-4.048321e-01,-4.707411e-01,-4.280748e-01,-3.731183e-01,-4.064173e-01
75%,-2.129559e-01,-2.378066e-02,-7.680507e-02,-2.523322e-01,-7.324915e-02,-6.182778e-02,-1.053093e-01,4.998466e-01
max,2.785333e+00,3.051851e+00,3.336086e+00,3.255164e+00,4.100416e+00,3.783766e+00,4.676995e+00,3.520726e+00


In [163]:
def k_means(n_clusters=3, n_inits=10, max_iter=500):
    kmeans = KMeans(
    init="random",
    n_clusters=n_clusters,
    n_init=n_inits,
    max_iter=max_iter,
    random_state=42)
    kmeans.fit(X)
    
    return kmeans

In [164]:
k_means_run = k_means()

In [165]:
kmeans_kwargs = {
    "init": "random",
    "n_init": 10,
    "max_iter": 300,
    "random_state": 42,
}

# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(X)
    sse.append(kmeans.inertia_)

In [166]:
source = pd.DataFrame({'y': sse, 'x': range(1, 11)})

In [167]:
chart_elbow = alt.Chart(source).mark_line().encode(
    alt.X('x:Q', title='Number of clusters'), 
    alt.Y('y:Q', title='SSE'))

In [168]:
kl = KneeLocator(
    range(1, 11), sse, curve="convex", direction="decreasing"
)

kl.elbow

6

In [169]:
# A list holds the silhouette coefficients for each k
silhouette_coefficients = []

# Notice you start at 2 clusters for silhouette coefficient
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(X)
    score = silhouette_score(X, kmeans.labels_)
    silhouette_coefficients.append(score)

In [170]:
source = pd.DataFrame({'y': silhouette_coefficients, 'x': range(2, 11)})

In [171]:
chart_silhouette = alt.Chart(source).mark_line().encode(
    alt.X('x:Q', title='Number of clusters'), 
    alt.Y('y:Q', title='Silhouette coefficients'))

In [172]:
chart_silhouette | chart_elbow

alt.HConcatChart(...)

In [173]:
k_means_run = k_means(n_clusters=6)

In [174]:
df_labels['clusters'] = k_means_run.labels_

In [175]:
dfs_kmeans = [pd.DataFrame(y) for x, y in df_labels.groupby('clusters', as_index=False)]

In [176]:
for i in range(0, len(dfs_kmeans)):
    num_items = len(dfs_kmeans[i])
    dfs_kmeans[i]['x'] = np.linspace(0.1, 3.0, num=num_items)
    dfs_kmeans[i]['y'] = np.linspace(0.1, 3.0, num=num_items)

In [177]:
def grid_photos(nx=6, ny=6, cluster=0, title='title'):
    
    nx, ny = (nx, ny)
    x = np.linspace(0, 1, nx)
    y = np.linspace(0, 1, ny)
    xv, yv = np.meshgrid(x, y)
    
    dfs_kmeans[cluster]['x'] = xv.ravel()[0:len(dfs_kmeans[cluster])]
    dfs_kmeans[cluster]['y'] = yv.ravel()[0:len(dfs_kmeans[cluster])]
    
    chart = alt.Chart(dfs_kmeans[cluster], title=title).mark_image(
        width=50,
        height=50
    ).encode(
        alt.X('x', axis=None),
        alt.Y('y', axis=None),
        url='url'
    )
    
    text = chart.mark_text(
    align='center',
    baseline='bottom',
    yOffset = -25
    #dx=3,  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        alt.Text('label')
    )
    
    return (chart + text)

In [178]:
cluster_0 = grid_photos(nx=8, ny=8, cluster=0, title='Cluster 1')
cluster_1 = grid_photos(nx=8, ny=8, cluster=1, title='Cluster 2')
cluster_2 = grid_photos(nx=9, ny=9, cluster=2, title='Cluster 3')
cluster_3 = grid_photos(nx=8, ny=8, cluster=3, title='Cluster 4')
cluster_4 = grid_photos(nx=8, ny=8, cluster=4, title='Cluster 5')
cluster_5 = grid_photos(nx=8, ny=8, cluster=5, title='Cluster 6')

In [179]:
((cluster_0 | cluster_1) & \
(cluster_2 | cluster_3) & \
(cluster_4 | cluster_5)).configure_axis(
        grid=False
    ).configure_view(
        strokeOpacity=0
    ).configure_concat(
    spacing=50
)

alt.VConcatChart(...)